In [1]:
!pip install emoji
import pandas as pd
import numpy as np
import re
import emoji
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 829.4 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234912 sha256=c931235ef97fe1247d9fc8219a2c4f79f89133a335810ffb68a98d41177b6ed7
  Stored in directory: /home/ati/.cache/pip/wheels/02/3d/88/51a592b9ad17e7899126563698b4e3961983ebe85747228ba6
Successfully built emoji


ModuleNotFoundError: No module named 'pandas'

In [ ]:
# loading the dataset
dataset = './train_hindi_mod.csv'
df1 = pd.read_csv(dataset)

en_hn_dict = './hindi_english_mod2.csv'
df_trans = pd.read_csv(en_hn_dict)

bad_words = './bad_words.csv'
with open(bad_words, 'r') as file:
    offensive_words = file.read().split(',')

# converting dataframe to dictionary
post_dict = df1.to_dict('records')
trans_dict = df_trans.to_dict('records')

: 

In [ ]:
# creating the dictionary for hindi to english
dictionary = {}

for row in trans_dict:
    if row['hindi'] not in dictionary:
        dictionary[row['hindi']] = [str(row['english'])]
    else:
        dictionary[row['hindi']].append(str(row['english']))
print("Dictionary size: ", len(dictionary))

: 

In [ ]:
# preprocessing the data
def preprocess_sentence(sentence):
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    
    # Tokenize the sentence
    tokens = word_tokenize(sentence.lower())
    
    # Remove stop words
    tokens = [token for token in tokens if token not in stop_words]
    
    # Stem each word
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    
    # Lemmatize each word
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in stemmed_tokens]
    
    # Join the tokens back into a sentence
    cleaned_sentence = ' '.join(lemmatized_tokens)
    
    return cleaned_sentence

: 

In [ ]:
def filtertext(sentnc):
    sentnc = re.sub(r'\bhttp\S+', '', sentnc) # Remove links
    # text = re.sub(r'\B#\w+', '', text) # Remove hashtags
    sentnc = re.sub(r'\B@\w+', '', sentnc) # Remove mentions
    sentnc = emoji.demojize(sentnc) # Replace emojis with text representation
    sentnc = re.sub(r':[a-z_]+:', '', sentnc) # Remove emoji codes
    words = re.split(r'[-, :0-9\n?\'\"]+', sentnc) # splitting based on delimiter
    words = list(filter(lambda word: word != "", words))
    sentnc = ' '.join(words)
    return sentnc

: 

In [ ]:
#translating the sentence
def translate(sentence):
    global dictionary
    words = sentence.split()
    new_sentence = ''
    for word in words:
        if word not in dictionary:
            new_sentence += word + ' '
        else:
            # for i in range(min(4, len(dictionary[word]))):
            for meaning in dictionary[word]:
                new_sentence += meaning + ' '
    return new_sentence.rstrip()


# translating and expanding the query
def transexpand(sentence):
    global dictionary
    words = sentence.split()
    new_sentence = ''
    for word in words:
        if word not in dictionary:
            new_sentence += word + ' '
        else:
            for i in range(min(4, len(dictionary[word]))):
                new_sentence += dictionary[word][0] + ' '
    return new_sentence.rstrip()

: 

In [ ]:
# splitting the data in test and train
X_train, X_test, y_train, y_test = train_test_split(df1['Post'], df1['Labels Set'], test_size=0.2, random_state= 9)

# filtering, translating and preprocessing the training data
X_train = X_train.apply(filtertext)
X_train = X_train.apply(translate)
X_train = X_train.apply(preprocess_sentence)


# initialising the lists and dicts
class_docs_totals = {} # to store how many docs a word appeared in a class
class_word_counts = {} # to sotre how many times a word appeared in a class
class_word_totals = {} # total no. of words in a class
class_doc_counts = {} # no. of doc in a class containing the word
class_priors = {} # probability of a class
vocab = set() # vocabulary generated from the training data

# print the training data metadata
count_1 = list(y_train).count(1)
count_0 = len(y_train) - count_1
print("Training data size: ", len(y_train))
print("Label Imbalance: ", count_1 / count_0)

: 

In [ ]:
# the method Used:  score = Summation of prob with prior
# Define a function to predict the class of a new text sample
def predict2(text):
    global class_docs_totals
    global class_word_counts
    global class_word_totals
    global class_doc_counts
    global class_priors
    global vocab
    text = filtertext(text)
    words = text.split()
    for word in words:
          if word in offensive_words:
              return 1
    text = transexpand(text)
    text = preprocess_sentence(text)
    probs = {}
    for c in np.unique(y_train):
        log_prob = np.log(class_priors[c])
        # log_prob = 1;
        for word in words:
          count = 1  # Laplace smoothing
          if word in vocab:
              count += class_doc_counts[c][word]
          log_prob += np.log(count / (class_docs_totals[c] + len(vocab)))
        probs[c] = log_prob
    # return max(probs, key=probs.get)
    if(probs[1] > 0.85 * probs[0]):
        return 1
    return 0

: 

In [ ]:
# the method Used:  score = Summation of prob with prior
# Define a function to predict the class of a new text sample
def predict3(text):
    global class_docs_totals
    global class_word_counts
    global class_word_totals
    global class_doc_counts
    global class_priors
    global vocab
    # text = preprocess_sentence(text)
    text = filtertext(text)
    words = text.split()
    for word in words:
          if word in offensive_words:
              return 1
    text = transexpand(text)
    text = preprocess_sentence(text)
    probs = {}
    for c in np.unique(y_train):
        log_prob = np.log(class_priors[c])
        # log_prob = 1;
        for word in words:
          count = 1  # Laplace smoothing
          if word in vocab:
              count += class_word_counts[c][word]
          log_prob += np.log(count / (class_word_totals[c] + len(vocab)))
        probs[c] = log_prob
    # return max(probs, key=probs.get)
    if(probs[1] > 0.85 * probs[0]):
        return 1
    return 0

: 

In [ ]:
# the method Used:  score = Summation of (1 + log(tf))
# Define a function to predict the class of a new text sample
def predict(text):
    global class_docs_totals
    global class_word_counts
    global class_word_totals
    global class_doc_counts
    global class_priors
    global vocab
    text = filtertext(text)
    words = text.split()
    for word in words:
          if word in offensive_words:
              return 1
    text = transexpand(text)
    text = preprocess_sentence(text)
    probs = {}
    for c in np.unique(y_train):
        score = 0
        for word in words:
          if word in vocab:
            if class_word_counts[c][word] != 0:
              score += (1 + np.log(class_word_counts[c][word]))
            else:
              score += 0
        probs[c] = score
    # return max(probs, key=probs.get)
    if(probs[1] > 0.85 * probs[0]):
        return 1
    return 0

: 

In [ ]:
def basemodel():
    global class_docs_totals
    global class_word_counts
    global class_word_totals
    global class_doc_counts
    global class_priors
    global vocab
    # evaluting the vocabulary
    for sentence in X_train:
        words = sentence.split()
        for word in words:
            vocab.add(word)

    print("Vocab size: ", len(vocab))

    # Create a dictionary to store the count of each word in each class
    # array of index = word for all labels
    # count how many time a word occured in each labels
    y_train_unique = np.unique(y_train)
    for c in y_train_unique:
        class_doc_counts[c] = {}
        class_word_counts[c] = {}
        for word in vocab:
            class_doc_counts[c][word] = 0
            class_word_counts[c][word] = 0

    # Count the number of occurrences of each word in each class
    for sent, c in zip(X_train, y_train):
        words = sent.split()
        for word in words:
            class_word_counts[c][word] += 1
        for word in np.unique(words):
            class_doc_counts[c][word] += 1

    for c in np.unique(y_train):
        class_priors[c] = (len(y_train[y_train == c]) + 1) / (len(y_train) + len(np.unique(y_train)))

    for c in np.unique(y_train):
        class_docs_totals[c] = sum(class_doc_counts[c].values())

    # Compute the total count of words in each class
    for c in y_train_unique:
        class_word_totals[c] = sum(class_word_counts[c].values())

: 

In [ ]:
basemodel()
correct = 0
tp = 0
fp = 0
tn = 0
fn = 0
for sen, off in zip(X_test, y_test):
    pred = predict(sen)
    if pred == 1:
      if(off == 1):
        tp += 1
        correct += 1
      else:
        fp += 1
    else:
      if(off == 0):
        correct += 1
        tn += 1
      else:
        fn += 1
accuracy = correct / len(y_test)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Accuracy:", accuracy)
print("Percision:", precision)
print("Recall:", recall)
f1_score = (2 * precision * recall) / (precision + recall)
print("F1 score:", f1_score)

: 

In [ ]:
print(predict('''#जीवनसंवाद: हम संघर्ष करना चाहते हैं, क्योंकि सामने संघर्ष की मिसाल रखना चाहते हैं. नए शहर में इतना कुछ नया होगा कि हमें अपनी जड़ें जमाने में बरसों बीत जाएंगे. हम कर्ज में जरूर हैं, लेकिन हम हारना नहीं चाहते. वह भी बिना लड़े.
#JeevanSamvad
@DayashankarMi
https://t.co/nd7troF0JF https://t.co/CmAzq2Mj8S'''))
print(predict('''मोदीजी और जब सारा देश सेना के साथ खडी है,पर दो सयाने विदेश मे पडे है?🤔 इसलिए बोलते हैं विदेशी मां का बेटा कभी देशभक्त न'''))
print(predict('''fuck her bastard'''))

: 

In [ ]:
sentnc = '''भारत ने कहा, चीन के सैनिकों के साथ लद्दाख में झड़प, क्या कहना है चीन का?
स्टोरी: टीम बीबीसी
आवाज़: भरत शर्मा https://t.co/Re6GyZVmbY'''
sentnc = filtertext(sentnc)
print(sentc)
sentnc = translate(sentnc)
print(sentnc)
sentnc = preprocess_sentence(sentnc)
print(sentnc)

: 

In [ ]:
0from textblob import TextBlob

def detect(query):
    """
    Detects offensive queries in Hindi using TextBlob.

    Args:
        query (str): The input query in Hindi.

    Returns:
        bool: True if the query is offensive, False otherwise.
    """
    # Create a TextBlob object for the input query
    blob = TextBlob(query)

    # Check for offensive language using TextBlob's sentiment polarity
    polarity = blob.sentiment.polarity

    # If polarity is less than 0, the query is considered offensive
    if polarity < 0:
        return 1
    else:
        return 0

correct = 0
tp = 0
fp = 0
tn = 0
fn = 0
for sen, off in zip(X_test, y_test):
    pred = max(predict(sen), detect(sen))
    if pred == 1:
      if(off == 1):
        tp += 1
        correct += 1
      else:
        fp += 1
    else:
      if(off == 0):
        correct += 1
        tn += 1
      else:
        fn += 1
accuracy = correct / len(y_test)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
print("Accuracy:", accuracy)
print("Percision:", precision)
print("Recall:", recall)
f1_score = (2 * precision * recall) / (precision + recall)
print("F1 score:", f1_score)

: 